<br />
<div align="center">
  <a href="https://deepwok.github.io/">
    <img src="../imgs/deepwok.png" alt="Logo" width="160" height="160">
  </a>

  <h1 align="center">Lab 4 for Advanced Deep Learning Systems (ADLS) - Hardware Stream</h1>

  <p align="center">
    ELEC70109/EE9-AML3-10/EE9-AO25
    <br />
		Written by
    <a href="https://aaron-zhao123.github.io/">Aaron Zhao, Pedro Gimenes </a>
  </p>
</div>

# General introduction

In this lab, you will learn how to emit SystemVerilog code for a neural network that's been transformed and optimized by MASE. Then, you'll design some hardware for a new Pytorch layer, and simulate the hardware using your new module.

# The Hardware Emit pass

The `emit_verilog` transform pass generates a top-level RTL file and testbench file according to the `MaseGraph`, which includes a hardware implementation of each layer in the network. This top-level file instantiates modules from the `components` library in MASE and/or modules generated using [HLS](https://en.wikipedia.org/wiki/High-level_synthesis), when internal components are not available. The hardware can then be simulated using [Verilator](https://www.veripool.org/verilator/), or deployed on an FPGA.

First, add Machop to your system PATH (if you haven't already done so) and import the required libraries.

In [1]:
import os, sys

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_cocotb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn

# TO DO: remove
# import os
# os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ["PATH"]
# !echo $PATH
!verilator

/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to debug


Usage:
        verilator --help
        verilator --version
        verilator --binary -j 0 [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --cc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --sc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --lint-only -Wall [source_files.v]...



Now, define the neural network. We're using a model which can be used to perform digit classification on the MNIST dataset.

In [2]:
class MLP(torch.nn.Module):
    """
    Toy FC model for digit recognition on MNIST
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(4, 10)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x = torch.nn.functional.relu(self.fc1(x))
        return x

Now, we'll generate a MaseGraph and add metadata. 

In [3]:
mlp = MLP()
mg = MaseGraph(model=mlp)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 1
x = torch.randn((batch_size, 2, 2))
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)

/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
DEBUG    graph():
    %x : [num_users=1] = placeholder[target=x]
    %flatten : [num_users=1] = call_function[target=torch.flatten](args = (%x,), kwargs = {start_dim: 1, end_dim: -1})
    %fc1 : [num_users=1] = call_module[target=fc1](arg

Before running `emit_verilog`, we'll quantize the model to fixed precision. Refer back to [lab 2](https://github.com/DeepWok/mase/blob/main/docs/labs/lab2.md) if you've forgotten how this works. Check that the data type for each node is correct after quantization.

In [4]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "machop",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"]["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 3]
    for result, result_info in node.meta["mase"]["common"]["results"].items():
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 3]

INFO     Inspecting graph [add_common_node_type_analysis_pass]
INFO     
Node name    Fx Node op     Mase type            Mase op      Value type
-----------  -------------  -------------------  -----------  ------------
x            placeholder    placeholder          placeholder  NA
flatten      call_function  implicit_func        flatten      float
fc1          call_module    module_related_func  linear       fixed
relu         call_function  module_related_func  relu         fixed
output       output         output               output       NA


At this point, it's important to run the `add_hardware_metadata` analysis pass. This adds all the required metadata which is later used by the `emit_verilog` pass, including:

1. The node's toolchain, which defines whether we use internal Verilog modules from the `components` library or the HLS flow.
2. The Verilog parameters associated with each node.

> **_TASK:_** Read [this page](https://jianyicheng.github.io/mase-tools/modules/analysis/add_metadata.html#add-hardware-metadata-analysis-pass) for more information on the hardware metadata pass.

In [5]:
mg, _ = add_hardware_metadata_analysis_pass(mg, None)

Finally, run the emit verilog pass to generate the SystemVerilog files.

In [6]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

INFO     Emitting Verilog...
INFO     Emitting internal components...


The generated files should now be found under `top/hardware`. 

> **_TASK:_** Read through `top/hardware/rtl/top.sv` and make sure you understand how our MLP model maps to this hardware design. 

You will notice the following instantiated modules:

* `fixed_linear`: this is found under `components/linear/fixed_linear.sv` and implements each Linear layer in the model.
* `fc<layer number>_weight/bias_source`: these are [BRAM](https://nandland.com/lesson-15-what-is-a-block-ram-bram/) memories which drive the weights and biases into the linear layers for computation.
* `fixed_relu`: found under `components/activations/fixed_relu.sv`, implements the ReLU activation.

As of now, we can't yet run a simulation on the model, as we haven't yet generated the memory components. To do this, run the `emit_bram` transform pass as follows, which will generate the memory initialization files and SystemVerilog modules to drive weights and biases into the linear layers. Finally, the `emit_verilog_tb` transform pass will generate the testbench files.


In [7]:
mg, _ = emit_bram_transform_pass(mg)

INFO     Emitting BRAM...
DEBUG    Emitting DAT file for node: fc1, parameter: weight
DEBUG    ROM module weight successfully written into /home/derek/.mase/top/hardware/rtl/fc1_weight_source.sv
DEBUG    Init data weight successfully written into /home/derek/.mase/top/hardware/rtl/fc1_weight_rom.dat
DEBUG    Emitting DAT file for node: fc1, parameter: bias
DEBUG    ROM module bias successfully written into /home/derek/.mase/top/hardware/rtl/fc1_bias_source.sv
DEBUG    Init data bias successfully written into /home/derek/.mase/top/hardware/rtl/fc1_bias_rom.dat


In [8]:
mg, _ = emit_cocotb_transform_pass(mg)

INFO     Emitting testbench...


> **_TASK:_** Now, you're ready to launch a simulation by calling the simulate action as follows.

In [9]:
from chop.actions import simulate

simulate(skip_build=False, skip_test=False)

/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/chop/actions/simulate.py:3: UserWarning: Python runners and associated APIs are an experimental feature and subject to change.
  from cocotb.runner import get_runner, get_results
%Warning-TIMESCALEMOD: /home/derek/mase/mase_components/activations/rtl/fixed_relu.sv:1:8: Timescale missing on this module as other modules have it (IEEE 1800-2017 3.14.2.3)
    1 | module fixed_relu #(
      |        ^~~~~~~~~~
                       /home/derek/.mase/top/hardware/rtl/top.sv:167:1: ... note: In file included from 'top.sv'
                       /home/derek/.mase/top/hardware/rtl/top.sv:8:8: ... Location of module with timescale
    8 | module top #(
      |        ^~~
                       ... For warning description see https://verilator.org/warn/TIMESCALEMOD?v=5.020
                       ... Use "/* verilator lint_off TIMESCALEMOD */" and lint_on around source to disable this message.


INFO: Running command perl /usr/local/bin/verilator -cc --exe -Mdir /home/derek/mase/docs/labs/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/cocotb/libs -L/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/cocotb/libs -lcocotbvpi_verilator' -Wno-fatal -Wno-lint -Wno-style -Wno-SELRANGE --trace -I/home/derek/.mase/top/hardware/rtl -I/home/derek/mase/mase_components/cast/rtl -I/home/derek/mase/mase_components/attention/rtl -I/home/derek/mase/mase_components/fixed_arith/rtl -I/home/derek/mase/mase_components/activations/rtl -I/home/derek/mase/mase_components/linear/rtl -I/home/derek/mase/mase_components/matmul/rtl -I/home/derek/mase/mase_components/binary_arith/rtl -I/home/derek/mase/mase_components/ViT/rtl -I/home/derek/mase/mase_components/conv/rtl -I/home/derek/mase/mase_components/sim/rtl -I/home/derek/mase/mase_components/common/rtl /home/derek/minicon

/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/_pytest/assertion/rewrite.py:186: DeprecationWarning: 'pipes' is deprecated and slated for removal in Python 3.13
  exec(co, module.__dict__)


     0.00ns INFO     cocotb.regression                  Found test mase_top_tb.test.test
     0.00ns INFO     cocotb.regression                  running test (1/1)


/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/einops/einops.py:827: DeprecationWarning: invalid escape sequence '\s'
  """
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/_pytest/assertion/rewrite.py:186: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  exec(co, module.__dict__)
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/einops/_torch_specific.py:108: ImportWarning: allow_ops_in_compiled_graph failed to import torch: ensure pytorch >=2.0
  warnings.warn("allow_ops_in_compiled_graph failed to import torch: ensure pytorch >=2.0", ImportWarning)
/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/_pytest/assertion/rewrite.py:186: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  exec(co, module.__dict__)
/home/derek/miniconda3/envs/mase/lib/python3.11/site

100000.00ns INFO     cocotb.regression                  test passed
100000.00ns INFO     cocotb.regression                  **************************************************************************************
                                                        ** TEST                          STATUS  SIM TIME (ns)  REAL TIME (s)  RATIO (ns/s) **
                                                        **************************************************************************************
                                                        ** mase_top_tb.test.test          PASS      100000.00          19.06       5245.48  **
                                                        **************************************************************************************
                                                        ** TESTS=1 PASS=1 FAIL=0 SKIP=0             100000.00          19.44       5144.74  **
                                                        ******************

/home/derek/miniconda3/envs/mase/lib/python3.11/tempfile.py:895: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp32wj2gs3wandb'>
  _warnings.warn(warn_message, ResourceWarning)
/home/derek/miniconda3/envs/mase/lib/python3.11/tempfile.py:895: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpq0lapna7wandb-artifacts'>
  _warnings.warn(warn_message, ResourceWarning)
/home/derek/miniconda3/envs/mase/lib/python3.11/tempfile.py:895: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpof_7jzemwandb-media'>
  _warnings.warn(warn_message, ResourceWarning)
/home/derek/miniconda3/envs/mase/lib/python3.11/tempfile.py:895: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpz0g_aqvlwandb-media'>
  _warnings.warn(warn_message, ResourceWarning)
/home/derek/miniconda3/envs/mase/lib/python3.11/tempfile.py:895: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp6i9clq_j'>
  _warnings.warn(warn_message, ResourceWa

- :0: Verilog $finish
INFO: Results file: /home/derek/mase/docs/labs/sim_build/results.xml


# Main Task

Pytorch has a number of layers which are available to users to define neural network models. At the moment, `emit_verilog` supports generating Verilog for models including Linear layers and the ReLU activation.

> **_MAIN TASK:_** choose another layer type from the [Pytorch list](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) and write a SystemVerilog file to implement that layer in hardware. Then, change the generated `top.sv` file to inject that layer within the design. For example, you may replace the ReLU activations with [Leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html#torch.nn.RReLU). Re-run the simulation and observe the effect on latency and accuracy.